In [1]:
import numpy as np
import pandas as pd


In [2]:
df=pd.read_csv('preprocessed.csv')
df.columns

/tmp/ipykernel_14299/869521615.py:1: DtypeWarning: Columns (0: invoice) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('preprocessed.csv')


Index(['invoice', 'stockcode', 'description', 'quantity', 'invoicedate',
       'price', 'customer_id', 'country', 'basket_size', 'basket_type',
       'invoice_value', 'product_type'],
      dtype='str')

In [3]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)   # remove numbers & symbols
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_desc'] = df['description'].astype(str).apply(clean_text)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1, 2),     # bigrams help a LOT (e.g. "tea light")
    min_df=5,               # ignore very rare words
    max_df=0.7              # ignore very common words
)

X = vectorizer.fit_transform(df['clean_desc'])


In [5]:
from sklearn.cluster import KMeans

inertia = []
k_range = range(8, 18)

for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X)
    inertia.append(km.inertia_)


In [6]:
k = 12  # example
kmeans = KMeans(n_clusters=k, random_state=42, n_init=20)
df['category_id'] = kmeans.fit_predict(X)


In [7]:
import numpy as np

terms = vectorizer.get_feature_names_out()

for i in range(k):
    center = kmeans.cluster_centers_[i]
    top_terms = [terms[ind] for ind in center.argsort()[-10:]]
    print(f"Cluster {i}: {top_terms}")


Cluster 0: ['wrap vintage', 'wrap christmas', 'roll', 'vintage', 'gift', 'vintage billboard', 'roll wrap', 'gift wrap', 'billboard', 'wrap']
Cluster 1: ['doilies', 'piece', 'tins', 'chain', 'chain kit', 'paper chain', 'kit', 'paper', 'vintage', 'set']
Cluster 2: ['wicker', 'white hanging', 'heart wicker', 'heart light', 'hanging heart', 'hanging', 'heart', 'light', 'light holder', 'holder']
Cluster 3: ['pack', 'decoration', 'small', 'design', 'card', 'blue', 'white', 'heart', 'pink', 'box']
Cluster 4: ['set', 'hand', 'warmer', 'hand warmer', 'set red', 'spotty', 'red spotty', 'retrospot', 'red retrospot', 'red']
Cluster 5: ['bag pink', 'charlotte', 'charlotte bag', 'bag red', 'design', 'lunch', 'lunch bag', 'jumbo bag', 'jumbo', 'bag']
Cluster 6: ['paisley cake', 'pack retrospot', 'teatime', 'teatime fairy', 'pack', 'fairy', 'fairy cake', 'cases', 'cake cases', 'cake']
Cluster 7: ['bakelike red', 'wall', 'diner', 'wall clock', 'diner wall', 'clock bakelike', 'bakelike', 'alarm', 'alarm